<a href="https://colab.research.google.com/github/Riddars/BeerProject/blob/comments_of_recipe/%D0%9F%D0%90%D0%A0%D0%A1%D0%98%D0%9D%D0%93.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import os

In [ ]:
data_df = pd.read_csv('https://raw.githubusercontent.com/Riddars/BearProject/main/recipeData.csv', encoding='ISO-8859-1')


In [ ]:
data_df.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz,116.0
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,955.0
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,NaN,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN,NaN
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,NaN,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,NaN
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0




---



In [ ]:
import re
import json


# Создайте словарь для хранения комментариев по ID рецепта
recipe_comments = {}

# Перебирайте каждую строку в датасете
for index, row in data_df.iterrows():
    recipe_id = row["BeerID"]
    url = row["URL"]
    full_url = "https://www.brewersfriend.com" + url
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    r = requests.get(full_url, headers=headers)

    if r.status_code == 200:
        soup = bs(r.text, "lxml")
        vacancies_names = soup.find_all('table', class_="bf_recipe_comment")

        comments_for_recipe = []  # Создайте список для хранения комментариев для данного рецепта

        for item in vacancies_names:
            comment_text = re.search(r'at \d{2}:\d{2}(?:am|pm)(.*)', item.text)
            if comment_text:
                clean_text = comment_text.group(1).strip()
                comments_for_recipe.append(clean_text)

        # Сохраните комментарии для данного рецепта в словаре
        recipe_comments[recipe_id] = comments_for_recipe

        # Сохраните словарь в файл после каждого обновления
        with open('recipe_comments.json', 'w') as file:
            json.dump(recipe_comments, file)